<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_glm_voice_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [ ]:
%cd /content/achatbot


In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [ ]:
!pip install -q "dist/achatbot-0.0.7.12-py3-none-any.whl[fastapi_bot_server,daily_room_audio_stream,livekit_room_audio_stream,silero_vad_analyzer,sense_voice_asr,llm_transformers_manual_voice_glm,bitsandbytes,queue]"


# download model

In [ ]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')
HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
!huggingface-cli download FunAudioLLM/SenseVoiceSmall  --local-dir ./models/FunAudioLLM/SenseVoiceSmall --local-dir-use-symlinks False

In [ ]:
# GLM-4-Voice-Tokenizer: Trained by adding vector quantization to the encoder part of Whisper,
# converting continuous speech input into discrete tokens.
# Each second of audio is converted into 12.5 discrete tokens.

!huggingface-cli download THUDM/glm-4-voice-tokenizer --local-dir ./models/THUDM/glm-4-voice-tokenizer --local-dir-use-symlinks False

In [ ]:
# GLM-4-Voice-9B: Pre-trained and aligned on speech modality based on GLM-4-9B,
# enabling understanding and generation of discretized speech.

!huggingface-cli download THUDM/glm-4-voice-9b --local-dir ./models/THUDM/glm-4-voice-9b --local-dir-use-symlinks False

In [ ]:
# GLM-4-Voice-Decoder: A speech decoder supporting streaming inference, retrained based on CosyVoice,
# converting discrete speech tokens into continuous speech output.
# Generation can start with as few as 10 audio tokens, reducing conversation latency.

!git lfs install
!git clone https://huggingface.co/THUDM/glm-4-voice-decoder ./models/THUDM/glm-4-voice-decoder

In [ ]:
#!huggingface-cli download THUDM/glm-4-voice-decoder --local-dir ./models/THUDM/glm-4-voice-decoder --local-dir-use-symlinks False

# test

In [ ]:
!cd /content/achatbot && python -m unittest test.integration.processors.test_glm_voice_processor.TestProcessor.test_audio


In [ ]:
!cd /content/achatbot && LOG_LEVEL=info GLM_VOICE_TYPE=text python -m unittest test.integration.processors.test_glm_voice_processor.TestProcessor.test_text


# run webrtc bot

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')
DAILY_API_KEY=userdata.get('DAILY_API_KEY')

LIVEKIT_URL=userdata.get('LIVEKIT_URL')
LIVEKIT_API_KEY=userdata.get('LIVEKIT_API_KEY')
LIVEKIT_API_SECRET=userdata.get('LIVEKIT_API_SECRET')


In [ ]:
%cd /content/achatbot

In [41]:
!cat /content/daily_glm_voice_bot.json


{
  "chat_bot_name": "DailyGLMVoiceBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "voice_llm": "glm"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "voice_llm": {
      "tag": "glm_voice_processor",
      "args": {
        "lm_gen_args": {
          "temperature": 0.2,
          "top_p": 0.8,
          "max_new_token": 2000
        },
        "voice_out_args": {
          "audio_sample_rate": 22050,
          "audio_channels": 1
        },
        "voice_tokenizer_path": "./models/THUDM/glm-4-voice-tokenizer",
        "model_path": "./models/THUDM/glm-4-voice-9b",
        "voice_decoder_path": "./models/THUDM/glm-4-voice-decoder",
        "torch_dtype": "auto",
        "bnb_quant_type": "int4",
        "device": "cuda"
      }
    }
  },


In [ ]:
!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_glm_voice_bot.json

In [42]:
!cat /content/daily_text_glm_voice_bot.json


{
  "chat_bot_name": "DailyAsrGLMVoiceBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "asr": "sense_voice",
    "voice_llm": "glm"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "asr": {
      "tag": "sense_voice_asr",
      "args": {
        "language": "zn",
        "model_name_or_path": "./models/FunAudioLLM/SenseVoiceSmall"
      }
    },
    "voice_llm": {
      "tag": "text_glm_voice_processor",
      "args": {
        "lm_gen_args": {
          "temperature": 0.2,
          "top_p": 0.8,
          "max_new_token": 2000
        },
        "voice_out_args": {
          "audio_sample_rate": 22050,
          "audio_channels": 1
        },
        "voice_tokenizer_path": "./models/THUDM/glm-4-voice-tokenizer",
        "model_path": "./mo

In [ ]:
!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_text_glm_voice_bot.json

# GLM-4-Voice

In [ ]:
!cd /content && rm -rf /content/GLM-4-Voice && git clone --recurse-submodules https://github.com/weedge/GLM-4-Voice.git


In [ ]:
%cd /content/GLM-4-Voice


In [ ]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')
HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!pip install -q gradio pyngrok

In [ ]:
!ngrok config add-authtoken $NGROK_TOKEN

In [ ]:
!cd /content/GLM-4-Voice && python model_server.py --host 0.0.0.0 --model-path /content/achatbot/models/THUDM/glm-4-voice-9b --port 10000 --dtype int4 --device cuda:0

In [ ]:
# open ngrok proxy url to run web gradio ui
!cd /content/GLM-4-Voice && python web_demo.py --port 8000 \
  --tokenizer-path  /content/achatbot/models/THUDM/glm-4-voice-tokenizer \
  --model-path /content/achatbot/models/THUDM/glm-4-voice-9b \
  --flow-path /content/achatbot/models/THUDM/glm-4-voice-decoder --ngrok

In [ ]:
# open ngrok proxy url to run web gradio ui
!cd /content/GLM-4-Voice && nohup python web_demo.py --tokenizer-path  /content/achatbot/models/THUDM/glm-4-voice-tokenizer \
  --model-path /content/achatbot/models/THUDM/glm-4-voice-9b \
  --flow-path /content/achatbot/models/THUDM/glm-4-voice-decoder --ngrok >nohup.out 2>&1 &

In [ ]:
!cat nohup.out

https://dashboard.ngrok.com/agents to stop net agent

# 优化：
- glm4-voice 生成的语音速度相对较慢，和单独使用CosyVoice出不多，取决于硬件来加速，或者工程上讲生成的文本和音频分开，并行生成（如果音频依赖问文本，则使用并发）
- 生成的音频开始有段卡顿音，需要去除
- glm4-voice 模型不支持语音中断, 不过工程上可以实现。